This notebook presents a way to parse Headhunter.ru storing information in ClickHouse Database. The list of queries is contained in "hh_data.csv". 

Script is adapted from source gained from https://LeftJoin.ru

In [3]:
#Required libraries to run ClickHouse
import requests
from datetime import datetime
import pandas as pd
import numpy as np
import re
import io
HOST = 'http://localhost:8123'

In [4]:
#Interaction with ClickHouse
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        data = r.text
    else:
        raise ValueError(r.text)   
    try:
        df = pd.read_csv(io.StringIO(data), sep = '\t')
    except:
        df = []
    return df

In [22]:
#ClickHouse interaction example
q = '''
    SHOW DATABASES
'''
get_clickhouse_df(q)

,INFORMATION_SCHEMA
0,default
1,headhunter
2,information_schema
3,system


In [23]:
#query to create database 'headhunter'vto store data in ClickHouse
#create_query = 
'''
CREATE DATABASE headhunter
'''
#query to create table 'vacancies_short'to store data in ClickHouse
#create_query = 
'''
CREATE TABLE headhunter.vacancies_short
(
    `added_at` DateTime,
    `query_string` String,
    `type` String,
    `level` String,
    `direction` String,
    `vacancy_id` UInt64,
    `premium` UInt8,
    `has_test` UInt8,
    `response_url` String,
    `address_city` String,
    `address_street` String,
    `address_building` String,
    `address_description` String,
    `address_lat` String,
    `address_lng` String,
    `address_raw` String,
    `address_metro_stations` String,
    `alternate_url` String,
    `apply_alternate_url` String,
    `department_id` String,
    `department_name` String,
    `salary_from` Nullable(Float64),
    `salary_to` Nullable(Float64),
    `salary_currency` String,
    `salary_gross` Nullable(UInt8),
    `name` String,
    `insider_interview_id` Nullable(UInt64),
    `insider_interview_url` String,
    `area_url` String,
    `area_id` UInt64,
    `area_name` String,
    `url` String,
    `published_at` DateTime,
    `employer_url` String,
    `employer_alternate_url` String,
    `employer_logo_urls_90` String,
    `employer_logo_urls_240` String,
    `employer_logo_urls_original` String,
    `employer_name` String,
    `employer_id` UInt64,
    `response_letter_required` UInt8,
    `type_id` String,
    `type_name` String,
    `archived` UInt8,
    `schedule_id` Nullable(String)
)
ENGINE = ReplacingMergeTree
ORDER BY vacancy_id
'''

'\nCREATE TABLE headhunter.vacancies_short\n(\n    `added_at` DateTime,\n    `query_string` String,\n    `type` String,\n    `level` String,\n    `direction` String,\n    `vacancy_id` UInt64,\n    `premium` UInt8,\n    `has_test` UInt8,\n    `response_url` String,\n    `address_city` String,\n    `address_street` String,\n    `address_building` String,\n    `address_description` String,\n    `address_lat` String,\n    `address_lng` String,\n    `address_raw` String,\n    `address_metro_stations` String,\n    `alternate_url` String,\n    `apply_alternate_url` String,\n    `department_id` String,\n    `department_name` String,\n    `salary_from` Nullable(Float64),\n    `salary_to` Nullable(Float64),\n    `salary_currency` String,\n    `salary_gross` Nullable(UInt8),\n    `name` String,\n    `insider_interview_id` Nullable(UInt64),\n    `insider_interview_url` String,\n    `area_url` String,\n    `area_id` UInt64,\n    `area_name` String,\n    `url` String,\n    `published_at` DateTime,\n

In [6]:
#Определение поисковых запросов
queries = pd.read_csv("./data/hh_data.csv")
queries

,Тип,Уровень,Направление,Ключевое слово
0,Профессия,Линейный специалист,Analytics,"""Marketing Analyst"" OR ""Маркетинговый аналитик"""
1,Профессия,Линейный специалист,Data Engineering,"""Инженер баз данных"" OR ""DWH Engineer"" OR ""Инж..."
2,Навык,NaN,Analytics,"""Python"" AND ""data"" AND ""анализ"""
3,Навык,NaN,BI,"""Tableau"""
4,Навык,NaN,Data Engineering,"""Etl"""
5,Навык,NaN,BI,"""Power BI"""
6,Навык,NaN,BI,"""d3.js"""
7,Навык,NaN,BI,"""Qlik"" OR ""Qlikview"" OR ""Qliksense"""
8,Навык,NaN,BI,"""Redash"""
9,Навык,NaN,BI,"""Дашборд"""


In [109]:
#Function to check for blacklisted vacancies
def check_name(name):
    """
    Check the argument to prevent blacklisted vacancies entering database
    """
#blacklisted vacancies
    bad_names = [r'курьер', r'грузчик', r'врач', r'менеджер по закупу',
           r'менеджер по продажам', r'оператор', r'повар', r'продавец',
          r'директор магазина', r'директор по продажам', r'директор по маркетингу',
          r'кабельщик', r'начальник отдела продаж', r'заместитель', r'администратор магазина', 
          r'категорийный', r'аудитор', r'юрист', r'контент', r'супервайзер', r'стажер-ученик', 
          r'су-шеф', r'маркетолог$', r'региональный', r'ревизор', r'экономист', r'ветеринар', 
          r'торговый', r'клиентский', r'начальник цеха', r'территориальный', r'переводчик', 
          r'маркетолог /', r'маркетолог по']
    for item in bad_names:
        if re.match(item, name):
            return True

In [110]:
#main cycle to form request, check results, fill database 

for query_type, level, direction, query_string in zip(queries['Тип'], queries['Уровень'], queries['Направление'], queries['Ключевое слово']):
    print(f'ключевое слово: {query_string}')
    url = 'https://api.hh.ru/vacancies'
    par = {'text': query_string, 'per_page':'10', 'page':0}
    #get number of web pages with the result of query
    r = requests.get(url, params=par).json()
    added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    pages = r['pages']
    found = r['found']
    vacancies_from_response = []
    #iterating query pages
    for i in range(0, pages + 1):
        par = {'text': query_string, 'per_page':'10', 'page':i}
        #filling vacancies_from_response list
        r = requests.get(url, params=par).json()
        try:
            
            vacancies_from_response.append(r['items'])        
        except Exception as E:
            continue
    #checking 'vacancies_from_response' for blacklisted vacancies 
    #as well as for vacancies that are already in database
    for item in vacancies_from_response:    
        for vacancy in item:
            query_test = f"SELECT count(1) FROM headhunter.vacancies_short WHERE vacancy_id={vacancy['id']} AND query_string='{query_string}'"
            if get_clickhouse_df(query_test, host = HOST, connection_timeout = 1500).empty:
                name = vacancy['name'].replace("'","").replace('"','')
                if check_name(name):
                    continue
            # form buffer variables to form INSERT query
            vacancy_id = vacancy['id']
            is_premium = int(vacancy['premium'])
            has_test = int(vacancy['has_test'])
            response_url = vacancy['response_url']
            try:
                address_city = vacancy['address']['city']
                address_street = vacancy['address']['street']
                address_building = vacancy['address']['building']
                address_description = vacancy['address']['description']
                address_lat = vacancy['address']['lat']
                address_lng = vacancy['address']['lng']
                address_raw = vacancy['address']['raw']
                address_metro_stations = str(vacancy['address']['metro_stations']).replace("'",'"')
            except TypeError:
                address_city = ""
                address_street = ""
                address_building = ""
                address_description = ""
                address_lat = ""
                address_lng = ""
                address_raw = ""
                address_metro_stations = ""
            alternate_url = vacancy['alternate_url']
            apply_alternate_url = vacancy['apply_alternate_url']
            try:
                department_id = vacancy['department']['id']
            except TypeError as E:
                department_id = ""
            try:
                department_name = vacancy['department']['name'].replace("'","").replace('"','')
            except TypeError as E:
                department_name = ""
            try:
                salary_from = vacancy['salary']['from']
            except TypeError as E:
                salary_from = "cast(Null as Nullable(UInt64))"
            try:
                salary_to = vacancy['salary']['to']
            except TypeError as E:
                salary_to = "cast(Null as Nullable(UInt64))"
            try:
                salary_currency = vacancy['salary']['currency']
            except TypeError as E:
                salary_currency = ""
            try:
                salary_gross = int(vacancy['salary']['gross'])
            except TypeError as E:
                salary_gross = "cast(Null as Nullable(UInt8))"
            name = vacancy['name']
            try:
                insider_interview_id = vacancy['insider_interview']['id']
            except TypeError:
                insider_interview_id = "cast(Null as Nullable(UInt64))"
            try:
                insider_interview_url = vacancy['insider_interview']['url']
            except TypeError:
                insider_interview_url = ""
            area_url = vacancy['area']['url']
            area_id = vacancy['area']['id']
            area_name = vacancy['area']['name']
            url = vacancy['url']
            published_at = vacancy['published_at']
            published_at = datetime.strptime(published_at,'%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d %H:%M:%S')
            try:
                employer_url = vacancy['employer']['url']
            except Exception as E:
                print(E)
                employer_url = ""
            try:
                employer_alternate_url = vacancy['employer']['alternate_url']
            except Exception as E:
                print(E)
                employer_alternate_url = ""
            try:
                employer_logo_urls_90 = vacancy['employer']['logo_urls']['90']
                employer_logo_urls_240 = vacancy['employer']['logo_urls']['240']
                employer_logo_urls_original = vacancy['employer']['logo_urls']['original']
            except Exception as E:
                print(E)
                employer_logo_urls_90 = ""
                employer_logo_urls_240 = ""
                employer_logo_urls_original = ""
            employer_name = vacancy['employer']['name'].replace("'","").replace('"','')
            try:
                employer_id = vacancy['employer']['id']
            except Exception as E:
                print(E)
            response_letter_required = int(vacancy['response_letter_required'])
            type_id = vacancy['type']['id']
            type_name = vacancy['type']['name']
            is_archived = int(vacancy['archived'])
            try:
                schedule = vacancy['schedule']['id']
            except Exception as E:
                print(E)
                schedule = ''
            if schedule == 'flyInFlyOut':
                continue
    #prepare list of values to insert to ClickHouse

            vacancies_short_list = [added_at, query_string, query_type, level, direction, vacancy_id, is_premium, 
                                    has_test, response_url, address_city, address_street, address_building, 
                                    address_description, address_lat, address_lng, address_raw, 
                                    address_metro_stations, alternate_url, apply_alternate_url, department_id, 
                                    department_name, salary_from, salary_to, salary_currency, salary_gross, name,
                                    insider_interview_id, insider_interview_url, area_url, area_id, area_name, url, 
                                    published_at, employer_url, employer_alternate_url, employer_logo_urls_90, 
                                    employer_logo_urls_240, employer_logo_urls_original, 
                                    employer_name, employer_id, response_letter_required, type_id, type_name, 
                                    is_archived, schedule]
            for index, item in enumerate(vacancies_short_list):
                if item is None:
                    vacancies_short_list[index] = ""
            tuple_to_insert = tuple(vacancies_short_list)
            
            query_insert = f'INSERT INTO headhunter.vacancies_short VALUES {tuple_to_insert}'
            get_clickhouse_df(query_insert, host = HOST, connection_timeout = 1500)


ключевое слово: "Marketing Analyst" OR "Маркетинговый аналитик"
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


ключевое слово: "Qlik" OR "Qlikview" OR "Qliksense"
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
ключевое слово: "Redash"
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not s

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
ключевое слово: "Data Science Director" OR "Директор по большим данным" OR "Директор по машинному обучению" OR "Директор продвинутой аналитики" OR "Директор предиктивной аналитики" OR "Chief Data Officer"
'url'
'alternate_url'
'logo_urls'
'id'
ключевое слово: "Chief Business Intelligence" Officer OR "Директор по визуализации" OR "Директор по аналитической отчетности"
ключевое слово: "Руководитель отдела аналитики" OR "Head of analytics" OR "Head of BI"
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
ключевое слово: "Руководитель data science"
ключевое слово: "Руководитель н

ключевое слово: "Аналитик данных" OR "Data analyst"
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' o